# 수집된 뉴스 기사 및 댓글에 대한 감정 분석
## * Word2Vec
* 데이터 
> 2017년 12월 1일부터 2018년 2월 1일까지 63일간 [네이버](http://www.naver.com)와 [다음](http://www.daum.net)의 랭킹뉴스와 뉴스의 댓글을 크롤링함.

In [1]:
import pickle
import html
import multiprocessing
from collections import namedtuple, OrderedDict
import re
import sys
import os
from glob import glob
from numba import jit
import warnings

os.environ['KERAS_BACKEND']='tensorflow'

import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import pandas as pd

from gensim.models import Word2Vec, KeyedVectors
from gensim.models.doc2vec import TaggedDocument

from konlpy.utils import pprint

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, roc_curve,  accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale, MinMaxScaler, LabelEncoder
from sklearn.manifold import TSNE

import keras.backend.tensorflow_backend as K
from keras.preprocessing import sequence
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.layers import Input, Flatten, Dense, Embedding, embeddings, merge, Dropout, Activation,  LSTM, Bidirectional, SimpleRNN, GRU
from keras.layers.convolutional import Conv1D, Conv2D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import SpatialDropout1D
from keras.utils import np_utils
from tensorflow.python.client import device_lib
from keras.layers.merge import dot

import xgboost as xgb

import matplotlib.pyplot as plt

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

C:\Users\pc\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\pc\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import Database_Handler as dh
import Basic_Module as bm

In [3]:
from ckonlpy.tag import Twitter
from konlpy.tag import Mecab
ct = Twitter()
mecab = Mecab()

## Stopwords

In [4]:
stopwords = open('./data/stopwordsList.txt',encoding='utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

## Load Data

In [5]:
#Naver
naverData = pickle.load(open('./data/pre_data/stastics/for_statistics_Naver_from_mongodb.pickled','rb'))
naverData = pd.DataFrame.from_dict(naverData, orient = 'index')
naverData.reset_index(inplace = True)
naverData.rename(columns = {'index' : 'id'}, inplace = True)
#Daum
daumData = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled','rb'))
daumData = pd.DataFrame.from_dict(daumData, orient = 'index')
daumData.reset_index(inplace = True)
daumData.rename(columns = {'index' : 'id'}, inplace = True)

print ('Naver : {}'.format(naverData.shape))
print ('Daum : {}'.format(daumData.shape))

Naver : (15120, 11)
Daum : (9372, 11)


In [6]:
extNaverData = naverData.loc[:, ['id', 'title', 'date', 'press', 'rank']].copy()
extNaverData['site'] = pd.Series(['Naver']*extNaverData.shape[0])
extDaumData = daumData.loc[:, ['id', 'title', 'date', 'press', 'rank']].copy()
extDaumData['site'] = pd.Series(['daum']*extDaumData.shape[0])

In [7]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/model/'
    classifierPath = '/Volumes/disk1/data/pre_data/classifier/'
    #newsPath = '/Volumes/data/pre_data/news_sentiment/'
    newsPath = './data/pre_data/news_sentiment/'
elif sys.platform =='win32':
    loadModelPath = 'd:/model/'
    classifierPath = 'd:/data/pre_data/classifier/'
    newsPath = './data/pre_data/news_sentiment/'

## TaggedDocument

In [8]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags sentiment')

## Word2Vec Model

### Twitter

In [9]:
taggerName = 'ct'

#### Train data set으로부터 TF-IDF Vectorizer을 만듦

In [10]:
trainName = './data/pre_data/train_test_Data/pre_data_train_for_word2vec_sentiment_by_ct.pickled'
train = pickle.load(open(trainName, 'rb'))
tfidf = bm.Build_tfidf(train)
del train

100%|██████████| 442359/442359 [00:00<00:00, 1429420.74it/s]


(442359, 159113)
vocab size : 159113


#### News to tagged Document

In [11]:
if os.path.isfile('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_word2vec_sentiment_analysis.pickled'):
    taggedDaumData = pickle.load(open('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_word2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedDaumData = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, ct, stopwords, 'daum')
    pickle.dump(taggedDaumData, open('./data/pre_data/tagged_data/pre_data_daum_news_by_ct_for_word2vec_sentiment_analysis.pickled', 'wb'))

    
if os.path.isfile('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_word2vec_sentiment_analysis.pickled'):
    taggedNaverData = pickle.load(open('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_word2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedNaverData = bm.MakeTaggedDataDAUM2(naverData, TaggedDocument, ct, stopwords, 'naver')
    pickle.dump(taggedNaverData, open('./data/pre_data/tagged_data/pre_data_naver_news_by_ct_for_word2vec_sentiment_analysis.pickled', 'wb'))

#### Load Model

In [12]:
model1 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-ct.model')
model2 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-ct.model')
model3 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-ct.model')

#### model1

In [13]:
modelName = bm.Return_ModelName('word2vec', model1,'ct')

In [14]:
classifierList = glob(classifierPath+'*'+modelName)

In [ ]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

LogisticRegression

In [ ]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedDaumData)

In [ ]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

In [ ]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedNaverData)

In [ ]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

In [ ]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model2

In [ ]:
modelName = bm.Return_ModelName('word2vec', model2,'ct')

In [ ]:
classifierList = glob(classifierPath+'*'+modelName)

In [ ]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

In [ ]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedDaumData)

In [ ]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

In [ ]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedNaverData)

In [ ]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

In [ ]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model3

In [ ]:
modelName = bm.Return_ModelName('word2vec', model3,'ct')

In [ ]:
classifierList = glob(classifierPath+'*'+modelName)

In [ ]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

In [ ]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedDaumData)

In [ ]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

In [ ]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedNaverData)

In [ ]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

In [ ]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

### Mecab

In [ ]:
taggerName = 'mecab'

#### Train data set으로부터 TF-IDF Vectorizer을 만듦

In [ ]:
trainName = './data/pre_data/train_test_Data/pre_data_train_for_word2vec_sentiment_by_mecab.pickled'
train = pickle.load(open(trainName, 'rb'))
tfidf = bm.Build_tfidf(train)
del train

#### News to tagged Document

In [ ]:
if os.path.isfile('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_word2vec_sentiment_analysis.pickled'):
    taggedDaumData = pickle.load(open('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_word2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedDaumData = bm.MakeTaggedDataDAUM2(daumData, TaggedDocument, mecab, stopwords, 'daum')
    pickle.dump(taggedDaumData, open('./data/pre_data/tagged_data/pre_data_daum_news_by_mecab_for_word2vec_sentiment_analysis.pickled', 'wb'))

    
if os.path.isfile('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_word2vec_sentiment_analysis.pickled'):
    taggedNaverData = pickle.load(open('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_word2vec_sentiment_analysis.pickled', 'rb'))
else:
    taggedNaverData = bm.MakeTaggedDataDAUM2(naverData, TaggedDocument, mecab, stopwords, 'naver')
    pickle.dump(taggedNaverData, open('./data/pre_data/tagged_data/pre_data_naver_news_by_mecab_for_word2vec_sentiment_analysis.pickled', 'wb'))

#### Load Model

In [ ]:
model1 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-0_min_count-2_by-ct.model')
model2 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-0_cbow_mean-1_min_count-2_by-ct.model')
model3 = Word2Vec.load(loadModelPath+'word2vec_size-1000_epoch-20_window-10_negative-7_hs-0_sg-1_cbow_mean-0_min_count-2_by-ct.model')

#### model1

In [ ]:
modelName = bm.Return_ModelName('word2vec', model1,'mecab')

In [ ]:
classifierList = glob(classifierPath+'*'+modelName)

In [ ]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

In [ ]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedDaumData)

In [ ]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

In [ ]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model1, tfidf, 1000, taggedNaverData)

In [ ]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

In [ ]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model2

In [ ]:
modelName = bm.Return_ModelName('word2vec', model2,'mecab')

In [ ]:
classifierList = glob(classifierPath+'*'+modelName)

In [ ]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

In [ ]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedDaumData)

In [ ]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

In [ ]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model2, tfidf, 1000, taggedNaverData)

  0%|          | 156/162640 [00:00<01:45, 1545.05it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:06<00:00, 24211.07it/s]
0it [00:00, ?it/s]

running time : 0:00:06.728832
Vectorizing Data


6876it [05:03, 22.62it/s]

In [21]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 47154.38it/s]
15120it [00:00, 96135.47it/s] 
15120it [00:00, 361213.41it/s]


CPU times: user 5.47 s, sys: 2.57 s, total: 8.04 s
Wall time: 16.5 s


In [ ]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict

#### model3

In [15]:
modelName = bm.Return_ModelName('word2vec', model3,'mecab')

In [16]:
classifierList = glob(classifierPath+'*'+modelName)

In [17]:
loadClassifierDict = dict(map(lambda x:bm.LoadClassifier(x), classifierList))

XGBoost
NeuralNetwork_2
NeuralNetwork_1
LogisticRegression
SVC
RandomForestClassifier


In [13]:
wv1, daum_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedDaumData)

  0%|          | 53/162640 [00:00<05:07, 529.39it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:07<00:00, 21965.46it/s]
0it [00:00, ?it/s]

running time : 0:00:07.474923
Vectorizing Data


9372it [04:22, 35.73it/s]


scaling Data
total running time : 0:04:32.074319


In [19]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_daum = dict(map(lambda x: bm.PredictSentiment(daum_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_daum = pd.DataFrame.from_dict(predictOutcome_daum)
predictOutcome_daum = extDaumData.merge(predictOutcome_daum,
                                   left_index = True, right_index = True)
predictOutcome_daum.to_csv('./outcome/outcome_news_sentiment_analysis_daum_'+modelName,index=None, encoding='utf-8')

9372it [00:00, 153254.52it/s]
9372it [00:00, 503438.96it/s]
9372it [00:00, 523610.58it/s]


CPU times: user 3.14 s, sys: 666 ms, total: 3.81 s
Wall time: 3.9 s


In [ ]:
wv1, naver_vecs_w2v = bm.Make_Pre_Data_For_DAUM(model3, tfidf, 1000, taggedNaverData)

  0%|          | 156/162640 [00:00<01:45, 1545.05it/s]

Word2Vec(vocab=162640, size=1000, alpha=0.025)


100%|██████████| 162640/162640 [00:06<00:00, 24211.07it/s]
0it [00:00, ?it/s]

running time : 0:00:06.728832
Vectorizing Data


6876it [05:03, 22.62it/s]

In [21]:
%%time
warnings.filterwarnings('ignore')
predictOutcome_naver = dict(map(lambda x: bm.PredictSentiment(naver_vecs_w2v, x, loadClassifierDict[x]), loadClassifierDict))
predictOutcome_naver = pd.DataFrame.from_dict(predictOutcome_naver)
predictOutcome_naver = extDaumData.merge(predictOutcome_naver,
                                   left_index = True, right_index = True)
predictOutcome_naver.to_csv('./outcome/outcome_news_sentiment_analysis_naver_'+modelName,index=None, encoding='utf-8')

15120it [00:00, 47154.38it/s]
15120it [00:00, 96135.47it/s] 
15120it [00:00, 361213.41it/s]


CPU times: user 5.47 s, sys: 2.57 s, total: 8.04 s
Wall time: 16.5 s


In [ ]:
del modelName
del predictOutcome_daum
del predictOutcome_naver
del daum_vecs_w2v
del naver_vecs_w2v
del wv1
del loadClassifierDict